#Crawling Data

Crawling data adalah sebuah proses pengambilan data dari suatu sumber internet. Dalam hal ini saya mencrawling data dari twitter menggunakan twint. Data yang saya peroleh adalah sebanyak 180 data. Data tersebut saya saring sehingga menjadi 101 data karena terdapat beberapa data yang tidak penting. Data tersebut merupakan tweet dari tanggal 19-09-2022 sampai 27-09-2022.

In [ ]:
%%capture
!git clone --depth=1 https://github.com/twintproject/twint.git
%cd twint
!pip3 install . -r requirements.txt

In [ ]:
%%capture
!pip install nest-asyncio

In [ ]:
%%capture
!pip install aiohttp==3.7.0

In [ ]:
import nest_asyncio
nest_asyncio.apply() #digunakan sekali untuk mengaktifkan tindakan serentak dalam notebook jupyter.
import twint #untuk import twint

In [ ]:
c = twint.Config()
c.Search = '#prabowo'
c.Pandas = True
c.Limit = 500
twint.run.Search(c)

1574837985727590400 2022-09-27 19:06:49 +0000 <guest_2024>  https://t.co/0KKdp6y1gK  #PemiluSerentak2024 #Jakarta #jawatengah #PuanMaharani #prabowo #indonesia2024 #NCTDREAM #budget2023ireland #JISOOxDiorPFW #ด้วยรักและหักหลังตอนจบ #audioleak #  https://t.co/xNmqP44MJB
1574829687570059264 2022-09-27 18:33:51 +0000 <guest_2024> Pada tahun 2024 akan di adakan pemilihan Presiden, diantara nama dibawah siapa yang akan anda pilih? #AniesBaswedan #Prabowo #Ganjar #PuanMaharani #PemiluSerentak2024
1574817998749110273 2022-09-27 17:47:24 +0000 <ER_R_OR> Semangat Juang #Ir_soekarno Ada Pada diri #prabowo
1574734945100173315 2022-09-27 12:17:22 +0000 <heraldindo> Fadli Zon memberikan bocoran mengenai kriteria calon wakil presiden (cawapres) Prabowo Subianto pada Pemilu Presiden (Pilpres) 2024. #FadliZon #Prabowo  https://t.co/lKBBHUZblj
1574729817483722752 2022-09-27 11:57:00 +0000 <RADARSOLO_> Ketua DPP PDIP Said Abdullah menanggapi isu Joko Widodo (Jokowi) yang akan diduetkan dengan Ketua Umum

In [ ]:
Tweets_df = twint.storage.panda.Tweets_df
Tweets_df["tweet"].to_csv("prabowo.csv")

#Preposesing

In [ ]:
%%capture
!pip install numpy

In [ ]:
%%capture
!pip install pandas

In [ ]:
%%capture
!pip install nltk

In [ ]:
%%capture
!pip install scikit-learn

In [ ]:
import numpy as np
#Library untuk mengelola data dalam Dataframe
import pandas as pd

#Lbrary untuk Preprocessing
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize # tokenizing

#Untuk membuat vektor dan TFIDF
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer

#Untuk melakukan proses SVD
from sklearn.decomposition import TruncatedSVD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
Tweets_df = pd.read_csv("https://raw.githubusercontent.com/maulidhan190081/file/main/prabowo2.csv")

Tweets_df

,tweet,label
0,Doa para kiai sepuh untuk Cak Imin dan Prabowo...,pro
1,Menteri Pertahanan (Menhan) Prabowo Subianto m...,pro
2,Semangat Juang #Ir_soekarno Ada Pada diri #pra...,pro
3,Negarawan sejati yang selalu ingin membawa ked...,pro
4,"""Zulfan Lindan bicara soal kepentingan negara ...",kontra
...,...,...
96,Masyarakat antusias menyambut kedatangan mente...,pro
97,"Anies Siap Maju di Pilpres 2024, Wagub Tak aka...",pro
98,Adian Napitupulu Sebut Presiden Jokowi Tidak A...,kontra
99,Elit Gerindra Desak Sandiaga Uno Mundur dari P...,kontra


In [ ]:
Tweets_df['tweet']=Tweets_df['tweet'].str.replace(',', '')
Tweets_df['tweet']=Tweets_df['tweet'].str.lower()
Tweets_df['tweet']

0      doa para kiai sepuh untuk cak imin dan prabowo...
1      menteri pertahanan (menhan) prabowo subianto m...
2      semangat juang #ir_soekarno ada pada diri #pra...
3      negarawan sejati yang selalu ingin membawa ked...
4      "zulfan lindan bicara soal kepentingan negara ...
                             ...                        
96     masyarakat antusias menyambut kedatangan mente...
97     anies siap maju di pilpres 2024 wagub tak akan...
98     adian napitupulu sebut presiden jokowi tidak a...
99     elit gerindra desak sandiaga uno mundur dari p...
100    buah duku buah kedondong 2024 gue prabowo dong...
Name: tweet, Length: 101, dtype: object

In [ ]:
#Untuk menghapus angka
import re

#Untuk menghilangkan Punctuation
import string

def remove_PTA_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
Tweets_df['tweet'] =Tweets_df['tweet'].apply(remove_PTA_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

Tweets_df['tweet'] = Tweets_df['tweet'].apply(remove_singl_char)

# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

Tweets_df['tweet'] = Tweets_df['tweet'].apply(word_tokenize_wrapper)
Tweets_df['tweet']

0      [doa, para, kiai, sepuh, untuk, cak, imin, dan...
1      [menteri, pertahanan, menhan, prabowo, subiant...
2           [semangat, juang, soekarno, ada, pada, diri]
3      [negarawan, sejati, yang, selalu, ingin, memba...
4      [zulfan, lindan, bicara, soal, kepentingan, ne...
                             ...                        
96     [masyarakat, antusias, menyambut, kedatangan, ...
97     [anies, siap, maju, di, pilpres, wagub, tak, a...
98     [adian, napitupulu, sebut, presiden, jokowi, t...
99     [elit, gerindra, desak, sandiaga, uno, mundur,...
100    [buah, duku, buah, kedondong, gue, prabowo, do...
Name: tweet, Length: 101, dtype: object

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Mendapatkan stopword indonesia
list_stopwords = stopwords.words('indonesian')

#Menghapus Stopword dari list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]
Tweets_df['tweet'] = Tweets_df['tweet'].apply(stopwords_removal)

Tweets_df['tweet']

0      [doa, kiai, sepuh, cak, imin, prabowo, amin, y...
1      [menteri, pertahanan, menhan, prabowo, subiant...
2                            [semangat, juang, soekarno]
3                [negarawan, sejati, membawa, kedamaian]
4      [zulfan, lindan, bicara, kepentingan, negara, ...
                             ...                        
96     [masyarakat, antusias, menyambut, kedatangan, ...
97     [anies, maju, pilpres, wagub, dukung, riza, pa...
98     [adian, napitupulu, presiden, jokowi, maju, pe...
99     [elit, gerindra, desak, sandiaga, uno, mundur,...
100    [buah, duku, buah, kedondong, gue, prabowo, ge...
Name: tweet, Length: 101, dtype: object

In [ ]:
Tweets_df.to_csv('TextPreprocessing.csv')

In [ ]:
dataTextPre = pd.read_csv('TextPreprocessing.csv')
vectorizer = CountVectorizer(min_df=1)
bag = vectorizer.fit_transform(dataTextPre['tweet'])

#TF

In [ ]:
print(vectorizer.vocabulary_)

{'doa': 124, 'kiai': 263, 'sepuh': 516, 'cak': 75, 'imin': 198, 'prabowo': 459, 'amin': 10, 'ya': 597, 'allah': 9, 'rabb': 473, 'alamin': 8, 'menteri': 346, 'pertahanan': 438, 'menhan': 339, 'subianto': 537, 'pujiannya': 471, 'presiden': 464, 'joko': 213, 'widodo': 594, 'jokowi': 214, 'mengakui': 327, 'kepemimpinan': 252, 'kenegarawanan': 249, 'semangat': 509, 'juang': 215, 'soekarno': 529, 'negarawan': 374, 'sejati': 502, 'membawa': 309, 'kedamaian': 233, 'zulfan': 603, 'lindan': 289, 'bicara': 65, 'kepentingan': 253, 'negara': 373, 'pilpres': 447, 'indonesia': 200, 'menurutnya': 347, 'capres': 78, 'figurnya': 153, 'menguntungkan': 336, 'as': 24, 'china': 85, 'rusia': 492, 'panglima': 391, 'tni': 564, 'ksad': 276, 'kompak': 271, 'salam': 497, 'komando': 268, 'muhaimin': 365, 'iskandar': 205, 'dekati': 97, 'puan': 468, 'ingatkan': 203, 'piagam': 445, 'koalisi': 265, 'id': 194, 'fakta': 150, 'berita': 55, 'terbaru': 549, 'yan': 598, 'permenas': 432, 'mandenas': 298, 'menyebut': 349, 'ra

In [ ]:
matrik_vsm=bag.toarray()
#print(matrik_vsm)
matrik_vsm.shape

(101, 605)

In [ ]:
matrik_vsm[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
a=vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(len(matrik_vsm[:,1]))
#dfb =pd.DataFrame(data=matrik_vsm,index=df,columns=[a])
dataTF =pd.DataFrame(data=matrik_vsm,index=list(range(1, len(matrik_vsm[:,1])+1, )),columns=[a])
dataTF

101


,abdurachman,adian,airlangga,airlanggaprabowo,aja,ajak,akhlak,al,alamin,allah,...,wilayah,wonderwoman,ya,yan,yatim,yusuf,zon,zonmenilai,zulfan,zuperrr
1,0,0,0,0,0,0,0,0,1,1,...,0,0,2,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
datalabel = pd.read_csv('https://raw.githubusercontent.com/maulidhan190081/file/main/prabowo2.csv')
datatwett = pd.concat([dataTF.reset_index(), datalabel["label"]], axis=1)
datatwett

,"(index,)","(abdurachman,)","(adian,)","(airlangga,)","(airlanggaprabowo,)","(aja,)","(ajak,)","(akhlak,)","(al,)","(alamin,)",...,"(wonderwoman,)","(ya,)","(yan,)","(yatim,)","(yusuf,)","(zon,)","(zonmenilai,)","(zulfan,)","(zuperrr,)",label
0,1,0,0,0,0,0,0,0,0,1,...,0,2,0,0,0,0,0,0,0,pro
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pro
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pro
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pro
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,kontra
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,97,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pro
97,98,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pro
98,99,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,kontra
99,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,kontra


In [ ]:
datatwett['label'].unique()

array(['pro', 'kontra'], dtype=object)

In [ ]:
datatwett.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Columns: 607 entries, ('index',) to label
dtypes: int64(606), object(1)
memory usage: 479.1+ KB


In [ ]:
### Train test split to avoid overfitting
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(datatwett.drop(labels=['label'], axis=1),
    datatwett['label'],
    test_size=0.3,
    random_state=0)

In [ ]:
X_train

,"(index,)","(abdurachman,)","(adian,)","(airlangga,)","(airlanggaprabowo,)","(aja,)","(ajak,)","(akhlak,)","(al,)","(alamin,)",...,"(wilayah,)","(wonderwoman,)","(ya,)","(yan,)","(yatim,)","(yusuf,)","(zon,)","(zonmenilai,)","(zulfan,)","(zuperrr,)"
80,81,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,92,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68,69,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
51,52,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,98,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67,68,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,65,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,48,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
from sklearn.feature_selection import mutual_info_classif
# determine the mutual information
mutual_info = mutual_info_classif(X_train, y_train)
mutual_info

array([1.26120086e-01, 0.00000000e+00, 7.31279931e-02, 0.00000000e+00,
       0.00000000e+00, 4.07808048e-03, 7.28394720e-02, 1.79334198e-02,
       0.00000000e+00, 7.72211477e-02, 1.24418753e-01, 0.00000000e+00,
       0.00000000e+00, 1.28147380e-03, 0.00000000e+00, 7.58395121e-02,
       0.00000000e+00, 5.18239457e-02, 0.00000000e+00, 5.88389736e-04,
       8.34147582e-03, 4.22145868e-02, 0.00000000e+00, 4.32077816e-02,
       0.00000000e+00, 0.00000000e+00, 6.69704557e-02, 0.00000000e+00,
       7.74395090e-02, 9.86351930e-02, 2.89320119e-02, 1.13172756e-01,
       6.95042571e-02, 1.00392294e-01, 5.00720468e-02, 5.90536546e-02,
       6.61250315e-02, 0.00000000e+00, 4.11165859e-02, 5.44128860e-02,
       1.83028240e-02, 4.72700685e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.20953633e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.40058196e-02, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.47581346e-02, 0.00000000e+00, 1.00337276e-02,
      

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

(penguatan,)     0.166279
(pertahanan,)    0.154974
(hubungan,)      0.152502
(pertemuan,)     0.148994
(menang,)        0.141868
                   ...   
(membawa,)       0.000000
(membahas,)      0.000000
(mellui,)        0.000000
(berjuang,)      0.000000
(menyambut,)     0.000000
Length: 606, dtype: float64

#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
testing=[]
for i in range(2,21):
  neigh = KNeighborsClassifier(n_neighbors=i)
  neigh.fit(X_train, y_train)
  Y_pred = neigh.predict(X_test) 
  testing.append(Y_pred)
testing

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.

[array(['pro', 'pro', 'pro', 'pro', 'pro', 'kontra', 'kontra', 'pro',
        'pro', 'kontra', 'kontra', 'kontra', 'kontra', 'kontra', 'kontra',
        'pro', 'pro', 'pro', 'pro', 'kontra', 'pro', 'kontra', 'kontra',
        'pro', 'kontra', 'kontra', 'kontra', 'kontra', 'pro', 'pro',
        'kontra'], dtype=object),
 array(['pro', 'pro', 'pro', 'pro', 'pro', 'kontra', 'pro', 'pro', 'pro',
        'pro', 'pro', 'kontra', 'pro', 'pro', 'kontra', 'pro', 'pro',
        'pro', 'pro', 'kontra', 'pro', 'kontra', 'pro', 'pro', 'kontra',
        'kontra', 'kontra', 'kontra', 'pro', 'pro', 'pro'], dtype=object),
 array(['pro', 'pro', 'pro', 'pro', 'pro', 'kontra', 'pro', 'pro', 'pro',
        'kontra', 'kontra', 'kontra', 'pro', 'kontra', 'kontra', 'pro',
        'pro', 'pro', 'pro', 'kontra', 'pro', 'kontra', 'pro', 'pro',
        'kontra', 'kontra', 'kontra', 'kontra', 'pro', 'pro', 'pro'],
       dtype=object),
 array(['pro', 'pro', 'pro', 'pro', 'pro', 'pro', 'pro', 'pro', 'pro',
        

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=14)
neigh.fit(X_train, y_train)
Y_pred = neigh.predict(X_test) 
Y_pred

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array(['pro', 'pro', 'pro', 'kontra', 'pro', 'pro', 'pro', 'pro',
       'kontra', 'pro', 'pro', 'pro', 'pro', 'pro', 'pro', 'pro', 'pro',
       'pro', 'pro', 'pro', 'pro', 'pro', 'pro', 'pro', 'pro', 'pro',
       'pro', 'pro', 'pro', 'pro', 'pro'], dtype=object)

In [ ]:
y_test

26     kontra
86        pro
2         pro
55     kontra
75        pro
94     kontra
16     kontra
73        pro
54     kontra
96        pro
53     kontra
93     kontra
78     kontra
13        pro
7      kontra
30        pro
22     kontra
24        pro
33     kontra
8         pro
43        pro
62        pro
3         pro
71        pro
45     kontra
48        pro
6      kontra
100       pro
82     kontra
76        pro
60        pro
Name: label, dtype: object

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score,precision_score
Y_pred = neigh.predict(X_test) 
accuracy_neigh=round(accuracy_score(y_test,Y_pred)* 100, 2)
acc_neigh = round(neigh.score(X_train, y_train) * 100, 2)
accuracy_neigh

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


61.29

In [ ]:
arre=[]
for i in range(len(testing)):
  testing[i][0] = neigh.predict(X_test) 
  accuracy_neigh=round(accuracy_score(y_test,testing[i][0])* 100, 2)
  acc_neigh = round(neigh.score(X_train, y_train) * 100, 2)
  arre.append(accuracy_neigh)
arre

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.

[61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29,
 61.29]

#KMeans

In [ ]:

from sklearn.cluster import KMeans

kmeans =KMeans(n_clusters=2)
kmeans=kmeans.fit(dataTF)
prediksi=kmeans.predict(dataTF)
centroids = kmeans.cluster_centers_
centroids

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  FutureWarning,


array([[ 0.00000000e+00,  0.00000000e+00,  1.38777878e-17, ...,
         1.00000000e-01,  0.00000000e+00,  0.00000000e+00],
       [ 1.09890110e-02,  1.09890110e-02,  9.89010989e-02, ...,
        -6.93889390e-18,  1.09890110e-02,  1.09890110e-02]])

In [ ]:
df= pd.DataFrame(prediksi)
df

,0
0,1
1,0
2,1
3,1
4,1
...,...
96,1
97,1
98,1
99,1


In [ ]:
centroid = pd.DataFrame(centroids)
centroid

,0,1,2,3,4,5,6,7,8,9,...,595,596,597,598,599,600,601,602,603,604
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,1.136364e-02,1.136364e-02,1.022727e-01,1.136364e-02,1.136364e-02,1.136364e-02,1.136364e-02,1.136364e-02,1.136364e-02,1.136364e-02,...,1.136364e-02,1.136364e-02,3.409091e-02,1.136364e-02,1.136364e-02,1.136364e-02,-6.938894e-18,-6.938894e-18,1.136364e-02,1.136364e-02
2,1.734723e-18,1.734723e-18,1.387779e-17,1.734723e-18,1.734723e-18,1.734723e-18,1.734723e-18,1.734723e-18,1.734723e-18,1.734723e-18,...,1.734723e-18,1.734723e-18,-6.938894e-18,1.734723e-18,1.734723e-18,1.734723e-18,9.090909e-02,9.090909e-02,1.734723e-18,1.734723e-18
